# ***Лабораторная работа №4***
# "Использование нейронных сетей для генерации текста"
# <i>Выполнил: Антипов Д.А. А-01-19</i>
# <i>Вариант 1</i>

## **Цель работы:**

получить практические навыки генерации текста.

## **Выполнение**

**1. Загрузить выборку стихотворений одного из поэтов в соответсвии с вариантом (нечетные - Маяковский).**

In [ ]:
import tensorflow as tf
from sklearn.model_selection  import train_test_split
import numpy as np
import pandas as pd
import os
import time

In [ ]:
device_name = tf.test.gpu_device_name()
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [ ]:
# Выбираем поэта
poet = 'mayakovskiy'

path_to_file = f'{poet}.txt'
path_to_file = tf.keras.utils.get_file(path_to_file, f'http://uit.mpei.ru/git/main/TDA/raw/branch/master/assets/poems/{path_to_file}')

1466243/1466243 [==============================] - 2s 1us/step


**2. Познакомиться с данными. Проанализировать статистические характеристики исходных данных(среднюю длину стихотворения, среднюю длину строки)**

In [ ]:
# Загружаем текст из файла.
# Стихотворения в файле разделены токеном '</s>' - сохраняем в переменную
with open(path_to_file,encoding = "utf-8") as f:
    text = f.read()

print(f'Length of text: {len(text)} characters')

EOS_TOKEN = '</s>'

Length of text: 815675 characters


In [ ]:
# Посмотрим на текст
print(text[:500])


Угрюмый дождь скосил глаза.
А за
решеткой
четкой
железной мысли проводов —
перина.
И на
нее
встающих звезд
легко оперлись ноги.
Но ги —
бель фонарей,
царей
в короне газа,
для глаза
сделала больней
враждующий букет бульварных проституток.
И жуток
шуток
клюющий смех —
из желтых
ядовитых роз
возрос
зигзагом.
За гам
и жуть
взглянуть
отрадно глазу:
раба
крестов
страдающе-спокойно-безразличных,
гроба
домов
публичных
восток бросал в одну пылающую вазу.

</s>


У —
лица.
Лица
у
догов
годов
рез —
че.
Че


In [ ]:
# Подсчет статистик

def mean_line_len(poem):
    lines = [len(line.strip()) for line in poem.split('\n') if len(line.strip())>0]
    return round(sum(lines)/len(lines))


def describe_poems(text,return_df = False):
    poems_list = [poem.strip() for poem in text.split(EOS_TOKEN) if len(poem.strip())>0]
    df = pd.DataFrame(data=poems_list,columns=['poem'])
    df['len'] = df.poem.map(len)
    df['lines'] = df.poem.str.count('\n')
    df['mean_line_len'] = df.poem.map(mean_line_len)
    if return_df:
        return df
    return df.describe()

In [ ]:
poem_df = describe_poems(text,return_df = True)
poem_df

,poem,len,lines,mean_line_len
0,Угрюмый дождь скосил глаза.\nА за\nрешеткой\nч...,449,34,12
1,У —\nлица.\nЛица\nу\nдогов\nгодов\nрез —\nче.\...,546,42,12
2,"«Какая очаровательная ночь!»\n«Эта,\n(указывае...",333,21,14
3,"Скрипка издергалась, упрашивая,\nи вдруг разре...",765,45,16
4,Войне ли думать:\n«Некрасиво в шраме»?\nЕй ли ...,901,69,14
...,...,...,...,...
738,Зеленые листики —\nи нет зимы.\nИдем\nраздолье...,279,24,10
739,"У меня растут года,\nбудет и семнадцать.\nГде ...",3656,280,13
740,"За море синеволное,\nза сто земель\nи вод\nраз...",666,49,12
741,Уважаемые\nтоварищи потомки!\nРоясь\nв сегодня...,3681,243,14


In [ ]:
poem_df.describe()

,len,lines,mean_line_len
count,743.000000,743.000000,743.000000
mean,1088.702557,81.532974,12.900404
std,623.199169,49.312455,2.407488
min,203.000000,19.000000,8.000000
25%,632.000000,44.000000,11.000000
50%,952.000000,72.000000,12.000000
75%,1345.500000,106.000000,15.000000
max,4172.000000,287.000000,18.000000


**3. Подготовить выборку для обучения**

In [ ]:
#Разбиваем данные на тренировочные, валидационные и тестовые

train_poems, test_poems = train_test_split(poem_df.poem.to_list(),test_size = 0.1, random_state = 42)
train_poems, val_poems = train_test_split(train_poems,test_size = 0.1, random_state = 42)

train_poems = f'\n\n{EOS_TOKEN}\n\n'.join(train_poems)
val_poems = f'\n\n{EOS_TOKEN}\n\n'.join(val_poems)
test_poems = f'\n\n{EOS_TOKEN}\n\n'.join(test_poems)

In [ ]:
# Словарь уникальных символов из текста.

vocab = sorted(set(text))+[EOS_TOKEN]
print(f'{len(vocab)} unique characters')
print (vocab)

140 unique characters
['\n', ' ', '!', '"', '%', '&', '(', ')', ',', '-', '.', '/', ':', ';', '<', '>', '?', 'A', 'B', 'C', 'D', 'E', 'G', 'H', 'I', 'J', 'K', 'M', 'N', 'O', 'P', 'R', 'S', 'U', 'V', 'X', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'w', 'z', '\xa0', '«', '»', 'à', 'ç', 'è', 'ö', 'ü', '̀', '́', '·', 'А', 'Б', 'В', 'Г', 'Д', 'Е', 'Ж', 'З', 'И', 'Й', 'К', 'Л', 'М', 'Н', 'О', 'П', 'Р', 'С', 'Т', 'У', 'Ф', 'Х', 'Ц', 'Ч', 'Ш', 'Щ', 'Ы', 'Ь', 'Э', 'Ю', 'Я', 'а', 'б', 'в', 'г', 'д', 'е', 'ж', 'з', 'и', 'й', 'к', 'л', 'м', 'н', 'о', 'п', 'р', 'с', 'т', 'у', 'ф', 'х', 'ц', 'ч', 'ш', 'щ', 'ъ', 'ы', 'ь', 'э', 'ю', 'я', 'ё', '–', '—', '’', '…', '№', '</s>']


In [ ]:
# Кодируем текст в виде числовой последовательности для подачи на вход нейронной сети

ids_from_chars = tf.keras.layers.StringLookup(
    vocabulary=list(vocab), mask_token=None)
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

def text_from_ids(ids):
    return tf.strings.reduce_join(chars_from_ids(ids), axis=-1).numpy().decode('utf-8')

def ids_from_text(text):
    return ids_from_chars(tf.strings.unicode_split(text, input_encoding='UTF-8'))

In [ ]:
# пример кодирования
ids = ids_from_text(train_poems[:20])
res_text = text_from_ids(ids)
print(train_poems[:20],ids,res_text,sep = '\n')

У Петровой
у Надежды
tf.Tensor(
[ 90   2  86 107 120 118 116 104 116 111   1 121   2  84 102 106 107 108
 106 129], shape=(20,), dtype=int64)
У Петровой
у Надежды


In [ ]:
# Кодируем данные и преобразуем их в Датасеты
train_ids = ids_from_text(train_poems)
val_ids = ids_from_text(val_poems)
test_ids = ids_from_text(test_poems)

train_ids_dataset = tf.data.Dataset.from_tensor_slices(train_ids)
val_ids_dataset = tf.data.Dataset.from_tensor_slices(val_ids)
test_ids_dataset = tf.data.Dataset.from_tensor_slices(test_ids)

In [ ]:
seq_length = 100
examples_per_epoch = len(train_ids_dataset)//(seq_length+1)

In [ ]:
train_sequences = train_ids_dataset.batch(seq_length+1, drop_remainder=True)
val_sequences = val_ids_dataset.batch(seq_length+1, drop_remainder=True)
test_sequences = test_ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in train_sequences.take(1):
  print(text_from_ids(seq))

У Петровой
у Надежды
не имеется одежды.
Чтоб купить
(пришли деньки!),
не имеется деньги́.
Ей
в расцве


In [ ]:
# Создаем датасет с input и target строками
# target сдвинута относительно input на один символ.
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [ ]:
split_input_target(list("Маяковский"))

(['М', 'а', 'я', 'к', 'о', 'в', 'с', 'к', 'и'],
 ['а', 'я', 'к', 'о', 'в', 'с', 'к', 'и', 'й'])

In [ ]:
train_dataset = train_sequences.map(split_input_target)
val_dataset = val_sequences.map(split_input_target)
test_dataset = test_sequences.map(split_input_target)

In [ ]:
for input_example, target_example in val_dataset.take(1):
    print("Input :", text_from_ids(input_example))
    print("Target:", text_from_ids(target_example))

Input : В смокинг вштопорен,
побрит что надо.
По гранд
по опере
гуляю грандом.
Смотрю
в антракте —
красавка 
Target:  смокинг вштопорен,
побрит что надо.
По гранд
по опере
гуляю грандом.
Смотрю
в антракте —
красавка н


In [ ]:
# Перемешиваем датасеты и разбиваем их на батчи для оптимизации обучения
# Batch size
BATCH_SIZE = 64

BUFFER_SIZE = 10000

def prepare_dataset(dataset):
    dataset = (
        dataset
        .shuffle(BUFFER_SIZE)
        .batch(BATCH_SIZE, drop_remainder=True)
        .prefetch(tf.data.experimental.AUTOTUNE))
    return dataset

train_dataset = prepare_dataset(train_dataset)
val_dataset = prepare_dataset(val_dataset)
test_dataset = prepare_dataset(test_dataset)

train_dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

**4. Построить нейронную сеть. Тип ячейки RNN выбрать в соответсвии с вариантом (LSTM)**

In [ ]:
# Длина словаря символов
vocab_size = len(vocab)

# размерность Embedding'а
embedding_dim = 256 #@param{type:"number"}

# Параметры RNN-слоя
rnn_units = 300 #@param {type:"number"}
dropout_p = 0.5

In [ ]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.lstm = tf.keras.layers.LSTM(rnn_units,
                                   dropout = dropout_p,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)

    if states is None:
        states = self.lstm.get_initial_state(x)

    x, *states = self.lstm(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [ ]:
model_5 = MyModel(
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [ ]:
# посмотрим на один батч из датасета
for input_example_batch, target_example_batch in train_dataset.take(1):
    example_batch_predictions = model_5(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 141) # (batch_size, sequence_length, vocab_size)


In [ ]:
example_batch_predictions[0][0]

<tf.Tensor: shape=(141,), dtype=float32, numpy=
array([ 1.6046757e-03,  2.5796483e-03, -1.5955156e-03, -6.6421466e-04,
       -4.8838546e-03,  4.7933506e-03,  1.9701705e-03,  4.8641930e-03,
        7.6674633e-03, -5.9319953e-03, -3.7517908e-03, -1.5375147e-03,
       -1.7565720e-03, -9.0143160e-04, -4.3582306e-03, -6.3046939e-03,
        1.9063733e-03,  6.7528249e-03, -4.3004849e-03, -2.6088553e-03,
       -1.4683431e-03,  1.8297128e-03,  9.6582025e-03, -4.2914250e-03,
       -7.4972410e-04,  6.4173471e-03,  4.6165832e-03,  1.9842413e-05,
        2.8695904e-03,  1.9777992e-03, -1.6086627e-03, -8.7593524e-03,
        1.6836994e-03,  4.7581512e-03, -4.5777418e-04, -3.9604022e-03,
       -4.3548364e-03, -6.9737206e-03, -5.1254506e-04, -5.8917870e-04,
       -3.4650380e-03,  1.6049956e-03, -7.3621566e-03, -6.0576452e-03,
        5.5510816e-03, -4.9042841e-03,  7.0305308e-04,  6.3169107e-04,
       -3.9747049e-04, -4.0708221e-03,  1.2660704e-02, -9.4204834e-03,
        1.2156089e-02,  9.569

In [ ]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()
sampled_indices

array([110,  40,   4, 125, 100,   4,  67, 125, 109,  34,   8, 126,  79,
       119, 102, 135,  56,   7,  82,  62,  89,  24, 125,  90,  72, 129,
        14,  70,  75,  20,  91,  80,  83,  79,  51,  24, 138,  67,  75,
       130, 134,   7,  65, 129, 115,   1, 101, 136, 123, 130,  20, 107,
        37, 140,  69,  75,  41,   3, 110, 119, 106,  45,  32,  77, 124,
        35, 111,  30,  39, 128,  50,   3, 115, 123,  40,  65, 112,  12,
       103,  86,  88,  58, 102, 137,   4,  55, 105, 132, 101,  84, 117,
         3,  60,  85,  35,  57,  83,  89, 101, 118])

In [ ]:
print("Input:\n", text_from_ids(input_example_batch[0]))
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices))

Input:
 ть.
Эй,
проснитесь, которые спят!
Разоблачай
с головы до пят.
Товарищ,
не смей молчать!

</s>

Много

Next Char Predictions:
 иc"чЮ"üчзU)шИса–u(Л»ТHчУБы;·ДCФЙМИoH…üДьё(èын
Я—хьCеZ</s>́Дd!исдhRЖцVйObъn!нхcèк/бПСwа’"tгюЯНп! ОVvМТЯр


**5. Обучить нейронную сеть на разных количествах эпох (5, 15, 30, 50, 70) при зафиксированных параметрах embedding_dim = 256, rnn_units = 300, T = 0.3 и сравнить результаты генерации (тексты), перплексию и статистические характеристики сгенерированных текстов. Выбрать оптимальное количество эпох**

In [ ]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [ ]:
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss)

Prediction shape:  (64, 100, 141)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(4.948912, shape=(), dtype=float32)


In [ ]:
print('perplexity: ',np.exp(example_batch_mean_loss))

perplexity:  141.02147


In [ ]:
model_5.compile(optimizer='adam', loss=loss)
model_5.summary()

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  36096     
                                                                 
 lstm (LSTM)                 multiple                  668400    
                                                                 
 dense (Dense)               multiple                  42441     
                                                                 
Total params: 746,937
Trainable params: 746,937
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    monitor="val_loss",
    save_weights_only=True,
    save_best_only=True)

In [ ]:
model_5 = MyModel(
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)
model_5.compile(optimizer='adam', loss=loss)


In [ ]:
EPOCHS_5 = 5
EPOCHS_15 = 15
EPOCHS_30 = 30
EPOCHS_50 = 50
EPOCHS_70 = 70

In [ ]:
# на 5 эпохах
history_5 = model_5.fit(train_dataset, validation_data = val_dataset, epochs=EPOCHS_5, callbacks=[checkpoint_callback])

Epoch 1/5
102/102 [==============================] - 8s 51ms/step - loss: 3.4207 - val_loss: 2.9701
Epoch 2/5
102/102 [==============================] - 4s 25ms/step - loss: 2.8098 - val_loss: 2.6927
Epoch 3/5
102/102 [==============================] - 3s 21ms/step - loss: 2.6550 - val_loss: 2.6008
Epoch 4/5
102/102 [==============================] - 3s 20ms/step - loss: 2.5869 - val_loss: 2.5438
Epoch 5/5
102/102 [==============================] - 4s 18ms/step - loss: 2.5384 - val_loss: 2.5041


In [ ]:
eval_loss = model_5.evaluate(test_dataset)
print('eval loss:',eval_loss)
print('perplexity',np.exp(eval_loss))

12/12 [==============================] - 0s 12ms/step - loss: 2.4997
eval loss: 2.499728202819824
perplexity 12.179183243139676


In [ ]:
# сбросим веса модели перед очередным обучением
model_5 = MyModel(
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)
model_5.compile(optimizer='adam', loss=loss)

In [ ]:
# на 15 эпохах
history_15 = model_5.fit(train_dataset, validation_data = val_dataset, epochs=EPOCHS_15, callbacks=[checkpoint_callback])

Epoch 1/15
102/102 [==============================] - 9s 45ms/step - loss: 3.4186 - val_loss: 2.9397
Epoch 2/15
102/102 [==============================] - 4s 23ms/step - loss: 2.7847 - val_loss: 2.6883
Epoch 3/15
102/102 [==============================] - 4s 26ms/step - loss: 2.6535 - val_loss: 2.6012
Epoch 4/15
102/102 [==============================] - 3s 19ms/step - loss: 2.5899 - val_loss: 2.5508
Epoch 5/15
102/102 [==============================] - 3s 19ms/step - loss: 2.5447 - val_loss: 2.5118
Epoch 6/15
102/102 [==============================] - 3s 17ms/step - loss: 2.5081 - val_loss: 2.4778
Epoch 7/15
102/102 [==============================] - 4s 23ms/step - loss: 2.4777 - val_loss: 2.4507
Epoch 8/15
102/102 [==============================] - 3s 17ms/step - loss: 2.4508 - val_loss: 2.4255
Epoch 9/15
102/102 [==============================] - 3s 17ms/step - loss: 2.4270 - val_loss: 2.4046
Epoch 10/15
102/102 [==============================] - 3s 18ms/step - loss: 2.4056 - val_lo

In [ ]:
eval_loss = model_5.evaluate(test_dataset)
print('eval loss:',eval_loss)
print('perplexity',np.exp(eval_loss))

12/12 [==============================] - 0s 12ms/step - loss: 2.2907
eval loss: 2.2907278537750244
perplexity 9.88212780820568


In [ ]:
# сбросим веса модели перед очередным обучением
model_5 = MyModel(
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)
model_5.compile(optimizer='adam', loss=loss)

In [ ]:
# на 30 эпохах
history_30 = model_5.fit(train_dataset, validation_data = val_dataset, epochs=EPOCHS_30, callbacks=[checkpoint_callback])

Epoch 1/30
102/102 [==============================] - 8s 45ms/step - loss: 3.4119 - val_loss: 2.9427
Epoch 2/30
102/102 [==============================] - 5s 27ms/step - loss: 2.7842 - val_loss: 2.6802
Epoch 3/30
102/102 [==============================] - 3s 19ms/step - loss: 2.6511 - val_loss: 2.5992
Epoch 4/30
102/102 [==============================] - 3s 19ms/step - loss: 2.5853 - val_loss: 2.5406
Epoch 5/30
102/102 [==============================] - 3s 18ms/step - loss: 2.5369 - val_loss: 2.5024
Epoch 6/30
102/102 [==============================] - 4s 22ms/step - loss: 2.5004 - val_loss: 2.4722
Epoch 7/30
102/102 [==============================] - 3s 17ms/step - loss: 2.4711 - val_loss: 2.4462
Epoch 8/30
102/102 [==============================] - 3s 17ms/step - loss: 2.4465 - val_loss: 2.4233
Epoch 9/30
102/102 [==============================] - 3s 19ms/step - loss: 2.4243 - val_loss: 2.4027
Epoch 10/30
102/102 [==============================] - 4s 19ms/step - loss: 2.4039 - val_lo

In [ ]:
eval_loss = model_5.evaluate(test_dataset)
print('eval loss:',eval_loss)
print('perplexity',np.exp(eval_loss))

12/12 [==============================] - 0s 12ms/step - loss: 2.1438
eval loss: 2.1438262462615967
perplexity 8.532020866332582


In [ ]:
# сбросим веса модели перед очередным обучением
model_5 = MyModel(
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)
model_5.compile(optimizer='adam', loss=loss)

In [ ]:
# на 50 эпохах
history_50 = model_5.fit(train_dataset, validation_data = val_dataset, epochs=EPOCHS_50, callbacks=[checkpoint_callback])

Epoch 1/50
102/102 [==============================] - 9s 51ms/step - loss: 3.4080 - val_loss: 2.9444
Epoch 2/50
102/102 [==============================] - 3s 23ms/step - loss: 2.7769 - val_loss: 2.6765
Epoch 3/50
102/102 [==============================] - 3s 20ms/step - loss: 2.6453 - val_loss: 2.5911
Epoch 4/50
102/102 [==============================] - 4s 22ms/step - loss: 2.5765 - val_loss: 2.5333
Epoch 5/50
102/102 [==============================] - 3s 21ms/step - loss: 2.5267 - val_loss: 2.4957
Epoch 6/50
102/102 [==============================] - 3s 20ms/step - loss: 2.4904 - val_loss: 2.4582
Epoch 7/50
102/102 [==============================] - 3s 17ms/step - loss: 2.4617 - val_loss: 2.4358
Epoch 8/50
102/102 [==============================] - 3s 18ms/step - loss: 2.4361 - val_loss: 2.4113
Epoch 9/50
102/102 [==============================] - 3s 17ms/step - loss: 2.4128 - val_loss: 2.3920
Epoch 10/50
102/102 [==============================] - 3s 20ms/step - loss: 2.3931 - val_lo

In [ ]:
eval_loss = model_5.evaluate(test_dataset)
print('eval loss:',eval_loss)
print('perplexity',np.exp(eval_loss))

12/12 [==============================] - 0s 11ms/step - loss: 2.0499
eval loss: 2.049909830093384
perplexity 7.767200706967358


In [ ]:
# сбросим веса модели перед очередным обучением
model_5 = MyModel(
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)
model_5.compile(optimizer='adam', loss=loss)

In [ ]:
# на 70 эпохах
history_70 = model_5.fit(train_dataset, validation_data = val_dataset, epochs=EPOCHS_70, callbacks=[checkpoint_callback])

Epoch 1/70
102/102 [==============================] - 9s 50ms/step - loss: 3.4129 - val_loss: 2.9523
Epoch 2/70
102/102 [==============================] - 4s 24ms/step - loss: 2.7891 - val_loss: 2.6851
Epoch 3/70
102/102 [==============================] - 3s 18ms/step - loss: 2.6549 - val_loss: 2.6047
Epoch 4/70
102/102 [==============================] - 4s 19ms/step - loss: 2.5939 - val_loss: 2.5559
Epoch 5/70
102/102 [==============================] - 3s 21ms/step - loss: 2.5511 - val_loss: 2.5159
Epoch 6/70
102/102 [==============================] - 3s 17ms/step - loss: 2.5165 - val_loss: 2.4865
Epoch 7/70
102/102 [==============================] - 3s 19ms/step - loss: 2.4883 - val_loss: 2.4617
Epoch 8/70
102/102 [==============================] - 3s 18ms/step - loss: 2.4636 - val_loss: 2.4397
Epoch 9/70
102/102 [==============================] - 3s 17ms/step - loss: 2.4423 - val_loss: 2.4202
Epoch 10/70
102/102 [==============================] - 3s 17ms/step - loss: 2.4218 - val_lo

In [ ]:
eval_loss = model_5.evaluate(test_dataset)
print('eval loss:',eval_loss)
print('perplexity',np.exp(eval_loss))

12/12 [==============================] - 0s 11ms/step - loss: 1.9966
eval loss: 1.9966121912002563
perplexity 7.364065744829364


**Результат**

| epochs | eval_loss | perplexity |
|--------|-----------|------------|
| 5      | 2.500     | 12.179     |
| 15     | 2.291     | 9.882      |
| 30     | 2.144     | 8.532      |
| 50     | 2.050     | 7.767      |
| 70     | 1.997     | 7.364      |

Судя по полученным результатам, оптимальное количество эпох соответствует 70 (epochs=70).

**6.Изменяя параметр температуры T проанализировать изменения сгенерированного текста. Выбрать оптимальное значение параметра.**

In [ ]:
# Создаем модель реализующую один шаг предсказания:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)


  # Этот фрагмент целиком написан с использованием Tensorflow, поэтому его можно выполнять
  # не с помощью интерпретатора языка Python, а через граф операций. Это будет значительно быстрее.
  # Для этого воспользуемся декоратором  @tf.function
  @tf.function
  def generate_one_step(self, inputs, states=None,temperature=1.0):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)


    # Return the characters and model state.
    return predicted_chars, states

In [ ]:
one_step_model = OneStep(model_5, chars_from_ids, ids_from_chars)

In [ ]:
T = 0.5 #@param {type:"slider", min:0, max:2, step:0.1}
N = 1000

start = time.time()
states = None
next_char = tf.constant(['\n'])
result = [next_char]

for n in range(N):
  next_char, states = one_step_model.generate_one_step(next_char, states=states,temperature=T)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()

result_text = result[0].numpy().decode('utf-8')
print(result_text)
print('_'*80)
print('\nRun time:', end - start)


и тебе
на только от держать в полед.
Гордо странами
не стоит под нам —
еще
не стоял на весь.
Мать бы —
развестите,
в копейках стройка,
в плеча
и в теле
на старом лотки.
Объедительство
в работать
в помня
старая торого.
Не верней восторга —
вы развернется
в ответ
за никой
с небесной строе последний.
И в сорок в колебе
солнце молодежения в пользуйте.
Должно оскально —
другой
под союзется надо —
чем полне под коммунизм.
Идите,
что же одна —
так свое согласенье —
против Иваности на требует по раскатал.
Ответственный
на слева день,
от завод
слова на полнула в хлебов.
Любовь
на примерит победину
все внимательной руких и другом.

</s>

О выходят миллионам
стальной картон.

</s>

Не стройте полеса,
что домом и кого я.
И верст всех делего —
как жизнь
показываются дело.

</s>

В как без разгода —
деловест протретив,
оконец и время,
обороненье к строга —
в оконок,
потымать по радостной,
а мерением выгод —
конец без страна просто.
На земле
не за комсомольцами ваши!

</s>

Страна не в коммуна карта

In [ ]:
describe_poems(result_text)

,len,lines,mean_line_len
count,5.000000,5.000000,5.0
mean,193.600000,10.200000,19.0
std,251.293255,15.385058,4.0
min,26.000000,0.000000,16.0
25%,36.000000,1.000000,17.0
50%,89.000000,4.000000,18.0
75%,189.000000,9.000000,18.0
max,628.000000,37.000000,26.0


**7.Проанализировать зависимость перплексии, скорости обучения, результатов генерации от параметров нейронной сети embedding_dim, rnn_units: embedding_dim = {vocab/4, vocab/2, vocab, vocab * 2, vocab * 4}, где vocab = размер словаря выборки. rnn_units = {10, 100, 300, 500}**

In [ ]:
# размерность Embedding'а
embedding_dim = [vocab_size//4, vocab_size//2, vocab_size, vocab_size*2, vocab_size*4]

# Параметры RNN-слоя
rnn_units = [10, 100, 300, 500]

In [ ]:
# embedding_dim = vocab_size//4
# rnn_units = 10

model_var_embed = MyModel(
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim[0],
    rnn_units=rnn_units[0])
model_var_embed.compile(optimizer='adam', loss=loss)

In [ ]:
# на 70 эпохах
history_70 = model_var_embed.fit(train_dataset, validation_data = val_dataset, epochs=EPOCHS_70, callbacks=[checkpoint_callback])

Epoch 1/70
102/102 [==============================] - 10s 63ms/step - loss: 4.4208 - val_loss: 3.7913
Epoch 2/70
102/102 [==============================] - 2s 12ms/step - loss: 3.6034 - val_loss: 3.5154
Epoch 3/70
102/102 [==============================] - 2s 7ms/step - loss: 3.4855 - val_loss: 3.4655
Epoch 4/70
102/102 [==============================] - 2s 11ms/step - loss: 3.4481 - val_loss: 3.4338
Epoch 5/70
102/102 [==============================] - 3s 16ms/step - loss: 3.4246 - val_loss: 3.4160
Epoch 6/70
102/102 [==============================] - 3s 10ms/step - loss: 3.4064 - val_loss: 3.3937
Epoch 7/70
102/102 [==============================] - 2s 8ms/step - loss: 3.3805 - val_loss: 3.3589
Epoch 8/70
102/102 [==============================] - 2s 7ms/step - loss: 3.3374 - val_loss: 3.3046
Epoch 9/70
102/102 [==============================] - 2s 7ms/step - loss: 3.2775 - val_loss: 3.2437
Epoch 10/70
102/102 [==============================] - 3s 11ms/step - loss: 3.2237 - val_loss:

In [ ]:
eval_loss = model_var_embed.evaluate(test_dataset)
print('eval loss:',eval_loss)
print('perplexity',np.exp(eval_loss))

12/12 [==============================] - 0s 5ms/step - loss: 2.6303
eval loss: 2.630324363708496
perplexity 13.878270779507359


In [ ]:
one_step_model = OneStep(model_var_embed, chars_from_ids, ids_from_chars)

T = 0.8
N = 1000

start = time.time()
states = None
next_char = tf.constant(['\n'])
result = [next_char]

for n in range(N):
  next_char, states = one_step_model.generate_one_step(next_char, states=states,temperature=T)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()

result_text = result[0].numpy().decode('utf-8')
print(result_text)
print('_'*80)
print('\nRun time:', end - start)


Ушимь мать —
бодковы.
Выминты,
преб — восёй обри.
Слам мацят ято
бов грибуй,
моеко.
Вымалдровюраристуши?
Не —
шыме.
Т Втачеки собитен,
в раро.
Лаз вота вала
с вщалей сесщара,
а пол и иб корси
узратузилиши,
зежареро вогов,
гапусекой,
вом дофета зотель опотий
ни поры
дулно
сюхь про сгуни —
сет,
в зестянича прурирак чедо старет,
на мам.
Пупод
вуз с несьмеки меливась раездавол,
габротом диси стега
пор прол зилевиана
пролых
вочки
скароз,
вел ни те те крестись
сох тае зла,
тему себева наой нувиты
и оте.
Ра к бодое.
Чомолщи на в сшееня,
и —
пола во —
в ригом нас,
—
е канарики рев гореложях в мазакна
наеми шедос
одут
гроргу
при
столе
вался.
Нествой
недой —
зае отознева тый дем звоюду».
Врит жвылы вопито.
Валин идасте потосн даннебь
гонет
«Скевасиких кынах стол —
рами огелеранесавна
вметол,
ла брочат баплить првем бомет сле невнивый бол птоль?
Евраци ред в кочель вомточесты тоши моколи пожтак.
с на дазкыт тый
ны чдиста
стобци етлает.
Зае нав гориче ствее,
потякь.
Сзласкожик.
Несь нав вомдать н

In [ ]:
# embedding_dim = vocab_size//2
# rnn_units = 10

model_var_embed = MyModel(
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim[1],
    rnn_units=rnn_units[0])
model_var_embed.compile(optimizer='adam', loss=loss)

In [ ]:
# на 70 эпохах
history_70 = model_var_embed.fit(train_dataset, validation_data = val_dataset, epochs=EPOCHS_70, callbacks=[checkpoint_callback])

Epoch 1/70
102/102 [==============================] - 9s 54ms/step - loss: 4.4006 - val_loss: 3.8024
Epoch 2/70
102/102 [==============================] - 2s 14ms/step - loss: 3.6085 - val_loss: 3.5138
Epoch 3/70
102/102 [==============================] - 2s 13ms/step - loss: 3.4812 - val_loss: 3.4586
Epoch 4/70
102/102 [==============================] - 2s 12ms/step - loss: 3.4410 - val_loss: 3.4310
Epoch 5/70
102/102 [==============================] - 3s 12ms/step - loss: 3.4188 - val_loss: 3.4079
Epoch 6/70
102/102 [==============================] - 2s 7ms/step - loss: 3.3939 - val_loss: 3.3814
Epoch 7/70
102/102 [==============================] - 2s 9ms/step - loss: 3.3652 - val_loss: 3.3467
Epoch 8/70
102/102 [==============================] - 2s 8ms/step - loss: 3.3255 - val_loss: 3.3006
Epoch 9/70
102/102 [==============================] - 2s 7ms/step - loss: 3.2747 - val_loss: 3.2483
Epoch 10/70
102/102 [==============================] - 2s 9ms/step - loss: 3.2237 - val_loss: 3

In [ ]:
eval_loss = model_var_embed.evaluate(test_dataset)
print('eval loss:',eval_loss)
print('perplexity',np.exp(eval_loss))

12/12 [==============================] - 0s 5ms/step - loss: 2.5933
eval loss: 2.593261480331421
perplexity 13.373317364323043


In [ ]:
one_step_model = OneStep(model_var_embed, chars_from_ids, ids_from_chars)

T = 0.5
N = 1000

start = time.time()
states = None
next_char = tf.constant(['\n'])
result = [next_char]

for n in range(N):
  next_char, states = one_step_model.generate_one_step(next_char, states=states,temperature=T)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()

result_text = result[0].numpy().decode('utf-8')
print(result_text)
print('_'*80)
print('\nRun time:', end - start)


пруште.
Дани —
в мераки со трони короебов толитей столони в сторет порни
наз подае повом сась невнини лони в брабось сторики,
налотей —
в гот ке ладо встастить
в кор —
в номе орет
голиму.
Что прадь вавазак
че трозать.
Зарась пет ворикить полеговня зариветь ма празого тулаки
бодать,
на в «Порять
нем ом пор —
к от стостинаванков разако,
бовом, —
сторь короте
в кориль гредаю в порем замом до в волистой
сно корем в на
помего
с борамни на под
делим
доторитыт
слот,
ни сторнух навет
сто воволь тет и в милуть,
в —
долонись —
заривет порах вобелистет,
в реболо
не ватовиней.
Одим, пробеннам
им нона помуще подий тростой скок позет че гостей
сто с в —
на на полекинатов
бесто поразнет
в нера в столь ос нин ното нергой колоно
подит.


С стого селегол под разды,
де важи в валь борето восту од тами.
Кориши пучий вростлит,
с —
полить поратой пок —
в продем —
содо кразась рыд нами вынок сторожа вореля бо наневы
на польсит,
на ма порабы с пиди,
в о раз к не на в полиник порет,
слосто воде в дерся на пор

In [ ]:
# embedding_dim = vocab_size
# rnn_units = 10

model_var_embed = MyModel(
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim[2],
    rnn_units=rnn_units[0])
model_var_embed.compile(optimizer='adam', loss=loss)

In [ ]:
# на 70 эпохах
history_70 = model_var_embed.fit(train_dataset, validation_data = val_dataset, epochs=EPOCHS_70, callbacks=[checkpoint_callback])

Epoch 1/70
102/102 [==============================] - 8s 48ms/step - loss: 4.3561 - val_loss: 3.7424
Epoch 2/70
102/102 [==============================] - 4s 19ms/step - loss: 3.5872 - val_loss: 3.5095
Epoch 3/70
102/102 [==============================] - 2s 12ms/step - loss: 3.4755 - val_loss: 3.4530
Epoch 4/70
102/102 [==============================] - 2s 9ms/step - loss: 3.4385 - val_loss: 3.4291
Epoch 5/70
102/102 [==============================] - 2s 9ms/step - loss: 3.4148 - val_loss: 3.4034
Epoch 6/70
102/102 [==============================] - 2s 7ms/step - loss: 3.3861 - val_loss: 3.3689
Epoch 7/70
102/102 [==============================] - 2s 12ms/step - loss: 3.3432 - val_loss: 3.3150
Epoch 8/70
102/102 [==============================] - 3s 10ms/step - loss: 3.2816 - val_loss: 3.2492
Epoch 9/70
102/102 [==============================] - 2s 7ms/step - loss: 3.2210 - val_loss: 3.1957
Epoch 10/70
102/102 [==============================] - 2s 7ms/step - loss: 3.1713 - val_loss: 3

In [ ]:
eval_loss = model_var_embed.evaluate(test_dataset)
print('eval loss:',eval_loss)
print('perplexity',np.exp(eval_loss))

12/12 [==============================] - 0s 5ms/step - loss: 2.5635
eval loss: 2.5635249614715576
perplexity 12.9814960337465


In [ ]:
one_step_model = OneStep(model_var_embed, chars_from_ids, ids_from_chars)

T = 0.5
N = 1000

start = time.time()
states = None
next_char = tf.constant(['\n'])
result = [next_char]

for n in range(N):
  next_char, states = one_step_model.generate_one_step(next_char, states=states,temperature=T)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()

result_text = result[0].numpy().decode('utf-8')
print(result_text)
print('_'*80)
print('\nRun time:', end - start)


Суторе в троду,
в обоми.
Не да свастяль бо не в ворака на в в сторовонь под растолься.
Не не дарев
стопра
сторовов ит гаривай валь
масторо в лез ит
о дота по трастем стодара стороть, строваров старанов посеть дени
несколь
б
на настой в не лись роскора на беровованит погуры в кране на вать сторовов не недро в торся сидем
и не горостно сулох и тоты скуротом.
На машелий это гроны всто тут гольной ворат
и порана
на ваторить тасть спорьния
в в стоновить зеговатной ботель в разегова
полох
я стотоли
процеколит вобероручся сдесноднай пок в прозогудорне то доголове
лаши сорамне,
в овери поливетрини
за крастом
за до недовини в дидям
и горожя
неба не дель,
пате и столи мерять мельноть потульно —
поласта стордый калькитной омены
короди итель перобо малим в пороном челься вод вобылька
мостить родя стоно траско
в кораста,
дористи в лече толь о о манит стодит
до просовы
до дуль неда
датле,
бомома,
на далот не дото бырия комы —
раздя палом овотом бероми котам полештей беланов не сторя в хогона,
полит

In [ ]:
# embedding_dim = vocab_size*2
# rnn_units = 10

model_var_embed = MyModel(
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim[3],
    rnn_units=rnn_units[0])
model_var_embed.compile(optimizer='adam', loss=loss)

In [ ]:
# на 70 эпохах
history_70 = model_var_embed.fit(train_dataset, validation_data = val_dataset, epochs=EPOCHS_70, callbacks=[checkpoint_callback])

Epoch 1/70
102/102 [==============================] - 8s 51ms/step - loss: 4.3157 - val_loss: 3.7009
Epoch 2/70
102/102 [==============================] - 3s 21ms/step - loss: 3.5671 - val_loss: 3.4978
Epoch 3/70
102/102 [==============================] - 3s 16ms/step - loss: 3.4667 - val_loss: 3.4495
Epoch 4/70
102/102 [==============================] - 2s 11ms/step - loss: 3.4346 - val_loss: 3.4241
Epoch 5/70
102/102 [==============================] - 2s 11ms/step - loss: 3.4112 - val_loss: 3.4010
Epoch 6/70
102/102 [==============================] - 3s 8ms/step - loss: 3.3814 - val_loss: 3.3591
Epoch 7/70
102/102 [==============================] - 2s 7ms/step - loss: 3.3323 - val_loss: 3.3038
Epoch 8/70
102/102 [==============================] - 2s 8ms/step - loss: 3.2694 - val_loss: 3.2365
Epoch 9/70
102/102 [==============================] - 2s 9ms/step - loss: 3.1995 - val_loss: 3.1632
Epoch 10/70
102/102 [==============================] - 2s 10ms/step - loss: 3.1200 - val_loss: 

In [ ]:
eval_loss = model_var_embed.evaluate(test_dataset)
print('eval loss:',eval_loss)
print('perplexity',np.exp(eval_loss))

12/12 [==============================] - 0s 5ms/step - loss: 2.5635
eval loss: 2.5635037422180176
perplexity 12.981220579013314


In [ ]:
one_step_model = OneStep(model_var_embed, chars_from_ids, ids_from_chars)

T = 0.5
N = 1000

start = time.time()
states = None
next_char = tf.constant(['\n'])
result = [next_char]

for n in range(N):
  next_char, states = one_step_model.generate_one_step(next_char, states=states,temperature=T)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()

result_text = result[0].numpy().decode('utf-8')
print(result_text)
print('_'*80)
print('\nRun time:', end - start)


мастрити на в поде керет
ташия
в спорика
голевак ток грова воро баброга.
Паде
уде колимашь побо — поли на зель,
прогрова красте горерень
карки прастодной в сторельки прастонты
пововый ноне
сшиной онинянь нат,
стрико.
Моста
порестерное содий грасте на стольелит не —
грабром
пелина не лем верит,
словели
растодо
прододил,
кроберонить
на ссетто вобной —
молака,
в —
бодан
и бодет болить дагорох
разна спорит нель натна дерки солости
в воскрабенить
и поритоды
ожарни —
овадонит
поликни,
бомали спазат
и на мы не мать
одит —
сомилет мазитот,
поскоредимама сестори
толоде,
в воликаго.
Стасто —
в и сверит в настить
кот порито стумы и воду
в воболь и сла пруд
и наб маркои достов
в кросто
и воте черну —
и востре бору сет страт
и скоборит банить и бом подат постоль,
кат в и о на в солит стут нитнели
выста разда стернох
и полестах
в помалать.

</s>



</s>

Сшебен
в растот в подей —
в оперо доне
не в татя настит
и пропонь
горену —
больчат масти бовебе —
не на стольты,
одань
на наболь лости,
на полькол

In [ ]:
# embedding_dim = vocab_size*4
# rnn_units = 10

model_var_embed = MyModel(
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim[4],
    rnn_units=rnn_units[0])
model_var_embed.compile(optimizer='adam', loss=loss)

In [ ]:
# на 70 эпохах
history_70 = model_var_embed.fit(train_dataset, validation_data = val_dataset, epochs=EPOCHS_70, callbacks=[checkpoint_callback])

Epoch 1/70
102/102 [==============================] - 10s 57ms/step - loss: 4.2686 - val_loss: 3.7092
Epoch 2/70
102/102 [==============================] - 3s 20ms/step - loss: 3.5821 - val_loss: 3.5116
Epoch 3/70
102/102 [==============================] - 2s 12ms/step - loss: 3.4758 - val_loss: 3.4547
Epoch 4/70
102/102 [==============================] - 2s 7ms/step - loss: 3.4393 - val_loss: 3.4290
Epoch 5/70
102/102 [==============================] - 3s 15ms/step - loss: 3.4043 - val_loss: 3.3850
Epoch 6/70
102/102 [==============================] - 3s 11ms/step - loss: 3.3637 - val_loss: 3.3409
Epoch 7/70
102/102 [==============================] - 2s 9ms/step - loss: 3.3094 - val_loss: 3.2771
Epoch 8/70
102/102 [==============================] - 2s 7ms/step - loss: 3.2491 - val_loss: 3.2249
Epoch 9/70
102/102 [==============================] - 2s 7ms/step - loss: 3.1922 - val_loss: 3.1610
Epoch 10/70
102/102 [==============================] - 2s 11ms/step - loss: 3.1213 - val_loss:

In [ ]:
eval_loss = model_var_embed.evaluate(test_dataset)
print('eval loss:',eval_loss)
print('perplexity',np.exp(eval_loss))

12/12 [==============================] - 0s 6ms/step - loss: 2.5523
eval loss: 2.55229115486145
perplexity 12.836480481187785


In [ ]:
one_step_model = OneStep(model_var_embed, chars_from_ids, ids_from_chars)

T = 0.5
N = 1000

start = time.time()
states = None
next_char = tf.constant(['\n'])
result = [next_char]

for n in range(N):
  next_char, states = one_step_model.generate_one_step(next_char, states=states,temperature=T)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()

result_text = result[0].numpy().decode('utf-8')
print(result_text)
print('_'*80)
print('\nRun time:', end - start)


настой.
Привели стозгорат выре от в столи то от памнана веревь
исторут воннён за и порось в от спродит —
на в просторать в либов на простотый та пора сностой в менать торь
жак в на сторнити
на,
нах достой на порялонный на селит.
Стото кодут нама гриколик востать вом на тадит в пронси кудной коралай стары —
в на непорем
не стовестой
не на с крома моко валове в в бы дут столикатер
не созеть.
и побат порнесь корна
сто в расевирит сто прунов возаники —
и велие —
са дрость моль и стохаки,
и там
в сторони,
калит в доратае,
за продит
на вет прусесь погирес,
на потитой отит слести
сорерие
сласчиви мимали —
в количный потом коме поздес,
кололий.

Потоники
на коря —
доря.
На пророденнат
на доны —
и настался порки рес —
перодажать
на престоли.
— На дометь трато мерако —
па слеченцел
на незели!

В и столице.
Чтараниньь тоты рестарска нанет ногорай погок слевого поделинать,
шта корка. —
В доли и порить не ста стаме жет порко насто олиздо
насет лебры поростой всенит че востай вырочи.
Простовит блим

| epochs | embedding_dim | rnn_units | eval_loss | perplexity |
|--------|---------------|-----------|-----------|------------|
| 70     | vocab_size//4 | 10        | 2.630     | 13.878     |
| 70     | vocab_size//2 | 10        | 2.593     | 13.373     |
| 70     | vocab_size    | 10        | 2.563     | 12.981     |
| 70     | vocab_size*2  | 10        | 2.563     | 12.981     |
| 70     | vocab_size*4  | 10        | 2.552     | 12.836     |

In [ ]:
# embedding_dim = vocab_size*4
# rnn_units = 100

model_var_rnn_units = MyModel(
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim[4],
    rnn_units=rnn_units[1])
model_var_rnn_units.compile(optimizer='adam', loss=loss)

In [ ]:
# на 70 эпохах
start = time.time()
history_70 = model_var_rnn_units.fit(train_dataset, validation_data = val_dataset, epochs=EPOCHS_70, callbacks=[checkpoint_callback])
end = time.time()
print('\nRun time:', end - start)

Epoch 1/70
102/102 [==============================] - 7s 38ms/step - loss: 3.5635 - val_loss: 3.0763
Epoch 2/70
102/102 [==============================] - 3s 19ms/step - loss: 2.9062 - val_loss: 2.7934
Epoch 3/70
102/102 [==============================] - 3s 16ms/step - loss: 2.7344 - val_loss: 2.6846
Epoch 4/70
102/102 [==============================] - 3s 17ms/step - loss: 2.6581 - val_loss: 2.6264
Epoch 5/70
102/102 [==============================] - 3s 14ms/step - loss: 2.6120 - val_loss: 2.5878
Epoch 6/70
102/102 [==============================] - 2s 11ms/step - loss: 2.5782 - val_loss: 2.5599
Epoch 7/70
102/102 [==============================] - 2s 11ms/step - loss: 2.5522 - val_loss: 2.5350
Epoch 8/70
102/102 [==============================] - 2s 13ms/step - loss: 2.5297 - val_loss: 2.5135
Epoch 9/70
102/102 [==============================] - 2s 12ms/step - loss: 2.5100 - val_loss: 2.4932
Epoch 10/70
102/102 [==============================] - 3s 13ms/step - loss: 2.4917 - val_lo

In [ ]:
eval_loss = model_var_rnn_units.evaluate(test_dataset)
print('eval loss:',eval_loss)
print('perplexity',np.exp(eval_loss))

12/12 [==============================] - 0s 8ms/step - loss: 2.1456
eval loss: 2.1456058025360107
perplexity 8.547217595305451


In [ ]:
one_step_model = OneStep(model_var_rnn_units, chars_from_ids, ids_from_chars)

T = 0.5
N = 1000

start = time.time()
states = None
next_char = tf.constant(['\n'])
result = [next_char]

for n in range(N):
  next_char, states = one_step_model.generate_one_step(next_char, states=states,temperature=T)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()

result_text = result[0].numpy().decode('utf-8')
print(result_text)
print('_'*80)
print('\nRun time:', end - start)


</s>

В богать на состит поднем,
настретитель
у верное
в обеда
под в коммунисториями —
под самой всему
под слова на последный будто
не теперь —
маличайте —
приметали в обратились стара.
Столько моем такие
на теперь
в это вот красней назовой.
Страники в просторой не летах
пропятный меня
в прополица в граничись
под стали
меня
в небо за просту ни от стран.

</s>

Рукой просло!
На от седите не домой.
Как под старитель
служай в откран.
В полобородими
в ного
сказал красный разверх.
А будет с бару
в большем вымерите,
а корот
запорет с простить.
Ленин —
по дерладище и старать на толгов.


Все сили
на поками
по во в свое
по картились
и выстритай в хазали разворде.
Столок заботорые самого ни вы покова.
На старительный мородиться картиться,
когда
признать зачень
под половодами.
А под нежно —
не раздало из коммунилось постариты.

</s>

Ты
рабочий стальный странцума.
Если всех в голово.
Помоганцы
и головочный поезда
и волнце в него летерами
на водайцами гряшей.
Дорогой обойде с мини́
поравестая
в 

In [ ]:
# embedding_dim = vocab_size*4
# rnn_units = 300

model_var_rnn_units = MyModel(
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim[4],
    rnn_units=rnn_units[2])
model_var_rnn_units.compile(optimizer='adam', loss=loss)

In [ ]:
# на 70 эпохах
start = time.time()
history_70 = model_var_rnn_units.fit(train_dataset, validation_data = val_dataset, epochs=EPOCHS_70, callbacks=[checkpoint_callback])
end = time.time()
print('\nRun time:', end - start)

Epoch 1/70
102/102 [==============================] - 9s 58ms/step - loss: 3.2658 - val_loss: 2.7926
Epoch 2/70
102/102 [==============================] - 4s 26ms/step - loss: 2.6915 - val_loss: 2.6196
Epoch 3/70
102/102 [==============================] - 3s 23ms/step - loss: 2.5860 - val_loss: 2.5441
Epoch 4/70
102/102 [==============================] - 4s 27ms/step - loss: 2.5255 - val_loss: 2.4919
Epoch 5/70
102/102 [==============================] - 3s 22ms/step - loss: 2.4808 - val_loss: 2.4557
Epoch 6/70
102/102 [==============================] - 4s 24ms/step - loss: 2.4468 - val_loss: 2.4249
Epoch 7/70
102/102 [==============================] - 3s 23ms/step - loss: 2.4171 - val_loss: 2.3997
Epoch 8/70
102/102 [==============================] - 4s 22ms/step - loss: 2.3914 - val_loss: 2.3770
Epoch 9/70
102/102 [==============================] - 3s 22ms/step - loss: 2.3685 - val_loss: 2.3530
Epoch 10/70
102/102 [==============================] - 3s 22ms/step - loss: 2.3479 - val_lo

In [ ]:
eval_loss = model_var_rnn_units.evaluate(test_dataset)
print('eval loss:',eval_loss)
print('perplexity',np.exp(eval_loss))

12/12 [==============================] - 0s 13ms/step - loss: 1.9745
eval loss: 1.9745146036148071
perplexity 7.203122435528266


In [ ]:
one_step_model = OneStep(model_var_rnn_units, chars_from_ids, ids_from_chars)

T = 0.5
N = 1000

start = time.time()
states = None
next_char = tf.constant(['\n'])
result = [next_char]

for n in range(N):
  next_char, states = one_step_model.generate_one_step(next_char, states=states,temperature=T)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()

result_text = result[0].numpy().decode('utf-8')
print(result_text)
print('_'*80)
print('\nRun time:', end - start)


выше открытый строгов.


. Первый
веков и советский радость.
Положил
столом ворот не сразу.
По самого кровью поля,
в коммунисты
в рабочий продавали в бородился —
с любители
клочка
в темного станки.
Смотрю,
с ноги соборовых всему.
Кровью потом.
Сомненьем в строчку.
Страна на бельсе.
«Что ж, говорит,
ни отец пором!
Не старается ли?
Вот в рядом высокий страны!
Приходите ли как в положи!
От весь пополнить за ногами лежат,
стройки,
как раз попросов на под ногами
тех,
но в коммунисты
на корабы
не подобные как с подымаемые красным масса.
С который фантили солнце —
на полицейский лаца.
В подняться в столом
провод стран
и придется
на заранут ворон.
И вот в пусть и стран.
Видно
сегодня
вставались
теперь
на хвост
прикончить —
как на первый разравления
в полетки волин.
И смерть
пополничной массы.
Вечер в кулака —
всех
с двигает торговой по конечного бедня,
советской облига.
Под носится года,
сквозь в каждого столицы.
Не дочина
в глаза —
одно последний попрос.
Не старом над болото и калеча.
Мальчи

In [ ]:
# embedding_dim = vocab_size*4
# rnn_units = 500

model_var_rnn_units = MyModel(
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim[4],
    rnn_units=rnn_units[3])
model_var_rnn_units.compile(optimizer='adam', loss=loss)

In [ ]:
# на 70 эпохах
start = time.time()
history_70 = model_var_rnn_units.fit(train_dataset, validation_data = val_dataset, epochs=EPOCHS_70, callbacks=[checkpoint_callback])
end = time.time()
print('\nRun time:', end - start)

Epoch 1/70
102/102 [==============================] - 10s 59ms/step - loss: 3.1799 - val_loss: 2.7425
Epoch 2/70
102/102 [==============================] - 6s 46ms/step - loss: 2.6489 - val_loss: 2.5749
Epoch 3/70
102/102 [==============================] - 5s 39ms/step - loss: 2.5377 - val_loss: 2.4955
Epoch 4/70
102/102 [==============================] - 6s 38ms/step - loss: 2.4722 - val_loss: 2.4383
Epoch 5/70
102/102 [==============================] - 5s 37ms/step - loss: 2.4222 - val_loss: 2.3954
Epoch 6/70
102/102 [==============================] - 5s 43ms/step - loss: 2.3806 - val_loss: 2.3558
Epoch 7/70
102/102 [==============================] - 5s 37ms/step - loss: 2.3443 - val_loss: 2.3220
Epoch 8/70
102/102 [==============================] - 5s 37ms/step - loss: 2.3094 - val_loss: 2.2920
Epoch 9/70
102/102 [==============================] - 6s 38ms/step - loss: 2.2792 - val_loss: 2.2626
Epoch 10/70
102/102 [==============================] - 5s 38ms/step - loss: 2.2503 - val_l

In [ ]:
eval_loss = model_var_rnn_units.evaluate(test_dataset)
print('eval loss:',eval_loss)
print('perplexity',np.exp(eval_loss))

12/12 [==============================] - 0s 19ms/step - loss: 1.9626
eval loss: 1.962598443031311
perplexity 7.117798249863704


In [ ]:
one_step_model = OneStep(model_var_rnn_units, chars_from_ids, ids_from_chars)

T = 0.5
N = 1000

start = time.time()
states = None
next_char = tf.constant(['\n'])
result = [next_char]

for n in range(N):
  next_char, states = one_step_model.generate_one_step(next_char, states=states,temperature=T)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()

result_text = result[0].numpy().decode('utf-8')
print(result_text)
print('_'*80)
print('\nRun time:', end - start)


Хоть выпускают в уши.
В наши ль,
не вывести
распластанный восторг.

</s>

Мы
было
собственный и зам,
в столе попробуй,
с тоскою уловических души на двадцать.

</s>

Товарищ,
платить по своем заколесами в город
в работу «Кромоде»
в комплекции
не слышу —
в звездно гроб,
за столица
в гробах волоса.
Комсомольцы,
вопрос не строй,
в пальцы
на борьбы
расставились трестов.
Виска —
не исполнил своих леса.
И просто —
не верную перемери.
А у без пересел и просто,
а не вы,
как под ногами
стали
на берегу в ремине.
Он — солнце
стоит руковичных облицей.
В глазах
распалял во все просторной,
на ней
носится
старатель
в делах вернель.


Вот это —
сегодня
собака трудовой спинными моря,
и не поповственных в столопал,
но только
слово не выставь, —
на любовном работе
в страже
старее
на столицей сплошном стих,
в страшном небе
не вылез ли
с половина стали.
Вся за домов
не отдал бы не выйдет положенно.
Копейки
в общее пугуются в руки.
На стене —
мысль-рокот.
Посудитель в мире —
это значит,
что в старою выружит

| epochs | embedding_dim | rnn_units | eval_loss | perplexity |
|--------|---------------|-----------|-----------|------------|
| 70     | vocab_size*4  | 10        | 2.552     | 12.836     |
| 70     | vocab_size*4  | 100       | 2.146     | 8.547      |
| 70     | vocab_size*4  | 300       | 1.974     | 7.203      |
| 70     | vocab_size*4  | 500       | 1.962     | 7.118      |

**Вывод:**

по результатам проведенных экспериментов можно сказать, что с ростом параметров embedding_dim и rnn_units показатели качества улучшаются, однако время обучения нейронной сети также возрастает. Сгенерированный текст становится более осмысленным, появляется все больше служебных символов, таких как точка, кавычки, запятые, тире и тд.